In [ ]:
R.home()

In [ ]:
library(tidyverse)
library(rjson)
library(Biostrings)

In [ ]:
# qc

In [ ]:
parse_fastp <- function(sample, path) {
    Ljs <- fromJSON(file=path)
    bind_cols(
        sample=sample,
        Ljs$summary$before_filtering %>% as_tibble %>% 
            select(readpair_raw=total_reads, q30_raw=q30_rate, R1_len_raw=read1_mean_length, R2_len_raw=read2_mean_length, GC_raw=gc_content),
        Ljs$summary$after_filtering %>% as_tibble %>% 
            select(readpair_flt=total_reads, q30_flt=q30_rate, R1_len_flt=read1_mean_length, R2_len_flt=read2_mean_length, GC_flt=gc_content)
    ) %>%
    mutate(readpair_raw=readpair_raw/2, readpair_flt=readpair_flt/2)
}

In [ ]:
TBqc <- snakemake@input$Lqc %>% map_dfr(
    ~parse_fastp(str_replace(basename(.x), '\\.\\w+$', ''), .x)
    )

In [ ]:
# align

In [ ]:
parse_align <- function(sample, path) {
    Vlog <- file(path, open="r") %>% readLines
    align_reads <- Vlog %>% grep('Mapped Reads: \\d+', ., value=TRUE) %>% 
        str_split(': ') %>% map_chr(~.x[2]) %>% as.double
    names(align_reads) <- sample
    return (align_reads)
}

In [ ]:
TBalign <- snakemake@input$Lalign_rate %>% 
    map(~parse_align(str_replace(basename(.x), '\\.\\w+$', ''), .x)) %>%
    unlist %>% as_tibble(rownames='sample') %>% dplyr::rename(align_reads=value)

In [ ]:
# dup

In [ ]:
parse_dup <- function (sample, path) {
    Vlog <- file(path, open="r") %>% readLines
    dup_reads <- Vlog %>% grep(' found \\d+ duplicates', ., value=TRUE) %>% 
        str_split(' ') %>% map_chr(~.x[4]) %>% as.double
    names(dup_reads) <- sample
    return (dup_reads)
}

In [ ]:
TBdup <- snakemake@input$Ldup_rate %>% 
    map(~parse_dup(str_replace(basename(.x), '\\.\\w+$', ''), .x)) %>%
    unlist %>% as_tibble(rownames='sample') %>% dplyr::rename(dup_reads=value)

In [ ]:
# variant info

In [ ]:
parse_altfreq <- function(path, high_altfreq_threshold=0.6) {
    tb <- read_tsv(path)
    high_altfreq <- tb %>% filter(ALT_FREQ > high_altfreq_threshold) %>% nrow / nrow(tb)
    sample <- unique(tb$REGION)
    names(high_altfreq) <- sample
    return (high_altfreq)
}

In [ ]:
high_altfreq_threshold <- snakemake@params$high_altfreq_threshold

In [ ]:
TBaltfreq <- snakemake@input$Lvariant_info %>% 
    map(~parse_altfreq(.x, high_altfreq_threshold)) %>% unlist %>% 
    as_tibble(rownames='sample') %>% dplyr::rename(high_altfreq=value)

In [ ]:
# nextclade

In [ ]:
parse_nextclade <- function(path) {
    tb <- read_delim(path, delim=';') %>% select(seqName, Nextclade_pango, partiallyAliased, clade_nextstrain, coverage, qc.overallScore, 
                  totalAminoacidSubstitutions, totalAminoacidDeletions, totalFrameShifts, 
                  aaSubstitutions, aaDeletions, missing)
    return (tb)
}

In [ ]:
TBnextclade <- parse_nextclade(snakemake@input$nextclade_csv) %>%
    dplyr::rename(sample=seqName)

In [ ]:
# merge

In [ ]:
TBres <- TBqc %>% left_join(TBdup, by='sample') %>% 
    left_join(TBalign, by='sample') %>% left_join(TBaltfreq, by='sample') %>%
    left_join(TBnextclade, by='sample')

In [ ]:
# format

In [ ]:
TBres <- TBres %>% mutate(
        qc.overallScore=round(qc.overallScore, 1),
        coverage=round(coverage, 3),
        align_rate=round(align_reads/(2*readpair_flt), 3),
        dup_rate=round(dup_reads/(2*readpair_flt), 3),
        high_altfreq=round(high_altfreq, 3),
        q30_raw=round(q30_raw, 3),
        q30_flt=round(q30_flt, 3),
        GC_raw=round(GC_raw, 3),
        GC_flt=round(GC_flt, 3),
        # PE150, and 30000 for nCoV genome length
        depth=round(readpair_flt * 300 * align_rate / 30000) %>% str_c('x')) %>% 
    select(-align_reads, -dup_reads)

In [ ]:
# relocate

In [ ]:
Vsel <- c('sample', 'Nextclade_pango', 'qc.overallScore', 'depth', 'coverage', 'align_rate', 'dup_rate', 'high_altfreq', 'totalFrameShifts', 'readpair_raw', 'q30_raw', 'R1_len_raw', 'R2_len_raw', 'GC_raw', 'readpair_flt', 'q30_flt', 'R1_len_flt', 'R2_len_flt', 'GC_flt', 'partiallyAliased', 'clade_nextstrain', 'totalAminoacidSubstitutions', 'totalAminoacidDeletions', 'aaSubstitutions', 'aaDeletions', 'missing')

In [ ]:
TBres <- TBres %>% select(all_of(Vsel))

In [ ]:
TBres %>% write_excel_csv(snakemake@output$upstream_stat)